# pyAFQ: Automated Fiber Quantification ... in Python

## Section 1: Four benefits of using pyAFQ compared to matlab AFQ:

### 1. Free and open source
    1. No need for a MATLAB license

### 2. Easier to run on cloud services
    1. For example, Amazon Web Services (AWS) or Microsoft Azure
    2. pyAFQ's only dependencies are other python packages which are installed automatically using "Package Installer for Python (pip)"
![](install_diagram.png)

### 3. Easier to run on BIDS datasets
1. The Brain Imaging Data Structure (BIDS) is a way of organizing neuroimaging and behavioral data
2. BIDS is a community effort
3. There are many [benefits to BIDS](https://bids.neuroimaging.io/benefits.html):
    1. It will be easy for another researcher to work on your data
        1. For example, by using BIDS you will save time trying to understand and reuse data acquired by a graduate student or postdoc that has already left the lab.
        
    2. There is a growing number of data analysis software that understand BIDS data, like pyAFQ
    
    3. Databases such as OpenNeuro.org, LORIS, COINS, XNAT, SciTran, and others will accept and export datasets organized according to BIDS.
        1. If you ever plan to share your data publicly (some journals require this) you can speed up the curation process by using BIDS.
        
    4. There are validation tools (also available online) that can check your dataset integrity and let you easily spot missing values.
    
![BIDS diagram](bids_diagram.png)

Image source: http://bids.neuroimaging.io/

### 4. Easier to customize methods and integrate with other pipelines
    1. pyAFQ provides tools to import tractography or mori group files from the original Matlab based AFQ to pyAFQ
    2. pyAFQ can use tractography from other pipelines like MRtrix3 or Qsiprep

## Section 2: Four pyAFQ code examples

### Tutorial 1: pyAFQ API 

In [ ]:
import plotly
from IPython.display import IFrame

from AFQ.api.group import GroupAFQ
import AFQ.data.fetch as afd

### Step 1: Organizing the data
In these examples we use the first subject and first session from the High angular resolution diffusion imaging ([HARDI](https://purl.stanford.edu/ng782rw8378)) dataset from Stanford’s Vista Lab

Anatomical data (anat) and Diffusion-weighted imaging data (dwi) are then extracted, formatted to be BIDS compliant, and placed in the AFQ data directory (by default in the users home directory) under:

AFQ_data/stanford_hardi/

This data represents the required preprocessed diffusion data necessary for intializing the GroupAFQ object. You will not need to run this function when you are using your own data. If your own data is already in BIDS, you do not need to perform this step!

In [ ]:
afd.organize_stanford_data(path="./my_example_dataset")

### Step 2: Initialize a GroupAFQ object
Creates a GroupAFQ object, that encapsulates tractometry. This object can be used to manage the entire AFQ pipeline, including:

    Tractography

    Registration

    Segmentation

    Cleaning

    Profiling

    Visualization

In [ ]:
myafq = GroupAFQ(bids_path='./my_example_dataset/stanford_hardi')

### Step 3: Run pyAFQ
After defining your pyAFQ API object, you can ask for the output of any step of the pipeline. It is common for users to just call export_all (for example, myafq.export_all()). However, if the user only wants the tractography, the user can instead call myafq.export(“streamlines”). In this case, we run myafq.export("all_bundles_figure") to export an interactive visualization of the tractometry results.

In [ ]:
bundle_html = myafq.export("all_bundles_figure")
plotly.io.show(bundle_html["01"][0])

### Tutorial 2: Looking at the results of CSD vs DTI
There are many choices of which methods to use when running a tracyometry pipeline. For example, to perform tractography, it is necessary to estimate a fiber orientation density function (fODF). pyAFQ makes it easy for users to choose which methods to use. Our default, used in Tutorial 1, is to use constrained spherical deconvolution (CSD). In this tutorial, we will use the diffusion tensor imaging (DTI) model for the fODF instead.

In [ ]:
myafq_dti = GroupAFQ(
    bids_path='./my_example_dataset/stanford_hardi',
    tracking_params={"odf_model": "DTI", "directions": "det"})

In [ ]:
# myafq_dti.export("indiv_bundles_figures")

IFrame(src="./my_example_dataset/stanford_hardi/derivatives/afq/sub-01/ses-01/viz_bundles/sub-01_ses-01_dwi_space-RASMM_model-DTI_desc-det-AFQ_ATR_L_viz.html", width=700, height=600)

In [ ]:
# myafq.export("indiv_bundles_figures")

IFrame(src="./my_example_dataset/stanford_hardi/derivatives/afq/sub-01/ses-01/viz_bundles/sub-01_ses-01_dwi_space-RASMM_model-CSD_desc-prob-AFQ_ATR_L_viz.html", width=700, height=600)

### Tutorial 3: Plotting Custom Bundles
pyAFQ is designed to be customizable. This example shows how you can customize it to define a new bundle based on both waypoint ROIs of your design, as well as endpoint ROIs of your design.

In these example, we run pyAFQ with both the custom ROIs and the default waypoint ROIs.

In [ ]:
from AFQ.api.group import GroupAFQ
import AFQ.api.bundle_dict as abd
import AFQ.data.fetch as afd
from AFQ.definitions.image import RoiImage

In [ ]:
afd.organize_stanford_data(path="./my_custom_bundles_dataset")

Custom bundles are defined by passing a custom bundle_info dictionary to AFQ.api.bundle_dict.BundleDict: The keys of bundle_info are bundle names; the values are another dictionary describing the bundle, with these key-value pairs:

- 'include' : a list of paths to Nifti files containing inclusion ROI(s).
  One must either have at least 1 include ROI, or 'start' or 'end' ROIs.
- 'exclude' : a list of paths to Nifti files containing exclusion ROI(s),
  optional.
- 'start' : path to a Nifti file containing the start ROI, optional
- 'end' : path to a Nifti file containing the end ROI, optional
- 'cross_midline' : boolean describing whether the bundle is required to
  cross the midline (True) or prohibited from crossing (False), optional.
  If None, the bundle may or may not cross the midline.
- 'space' : a string which is either 'template' or 'subject', optional

If this field is not given or 'template' is given, the ROI will be
transformed from template to subject space before being used.
- 'prob_map' : path to a Nifti file which is the probability map,
  optional.

In [ ]:
or_rois = afd.read_or_templates(as_img=False)
print(or_rois["left_OR_1"])

The cell above generates a dicitionary containing paths to Nifti files with our custom ROIs in them. 

In [ ]:
bundles = abd.BundleDict({
    "L_OR": {
        "include": [
            or_rois["left_OR_1"],
            or_rois["left_OR_2"]],
        "exclude": [
            or_rois["left_OP_MNI"],
            or_rois["left_TP_MNI"],
            or_rois["left_pos_thal_MNI"]],
        "start": or_rois['left_thal_MNI'],
        "end": or_rois['left_V1_MNI'],
        "cross_midline": False,
    },
    "R_OR": {
        "include": [
            or_rois["right_OR_1"],
            or_rois["right_OR_2"]],
        "exclude": [
            or_rois["right_OP_MNI"],
            or_rois["right_TP_MNI"],
            or_rois["right_pos_thal_MNI"]],
        "start": or_rois['right_thal_MNI'],
        "end": or_rois['right_V1_MNI'],
        "cross_midline": False
    }
})


It is easy to combine bundle dictionaries. Here, we add in the default bundles.

In [ ]:
bundles = bundles + abd.BundleDict()

The bundle dictionary can be passed to the GroupAFQ object. 

In [ ]:
my_afq_OR = GroupAFQ(
    bids_path="./my_custom_bundles_dataset/stanford_hardi",
    bundle_info=bundles,
    tracking_params={"n_seeds": 3, "seed_mask": RoiImage()})

In [ ]:
bundle_html = my_afq_OR.export("all_bundles_figure")
plotly.io.show(bundle_html["01"][0])

### Tutorial 4: A different approach to bundle recognition, RecoBundles

In [ ]:
import plotly

from AFQ.api.group import GroupAFQ
import AFQ.data.fetch as afd

In [ ]:
afd.organize_stanford_data(path="./my_recobundles_dataset")

RecoBundles a different way to recognize white matter bundles than AFQ.
RecoBundles uses bundle models as shape priors for detecting similar streamlines and bundles from the tractography, instead of AFQ which relies on regions of interest to recognize white matter bundles.

In [ ]:
myafq_reco = GroupAFQ(
    "./my_recobundles_dataset/stanford_hardi",
    segmentation_params={"seg_algo": "reco80"})

In [ ]:
bundle_html = myafq_reco.export("all_bundles_figure")
plotly.io.show(bundle_html["01"][0])

## Section 3: Five other features

### 1. We developed a python library called cloudknot (https://nrdg.github.io/cloudknot/) to run python code on amazon web services

### 2. Can be used as a reconstruction pipeline of "QSIprep"
        1. QSIprep is also free, open source, and uses BIDS.
        2. Includes automatically generated preprocessing pipelines that correctly group, distortion correct, motion correct, denoise, coregister and resample your scans, producing visual reports and QC metrics.
![](QSIprep.png)
Image source: https://qsiprep.readthedocs.io/en/latest/index.html

### 3. Can add custom tissue properties
    As an example, one might have “ICVF” and “ODI” maps in a BIDS pipeline named “noddi”:

        ~/my_bids_path/
        ├── dataset_description.json
        └── derivatives
            ├── noddi
            │   ├── dataset_description.json
            │   └── sub-01
            │       └── ses-01
            │           └── sub-01_ses-01_ICVF.nii.gz
            │           └── sub-01_ses-01_ODI.nii.gz
            └── vistasoft
                ├── dataset_description.json
                └── sub-01
                    └── ses-01
                        └── dwi
                            ├── sub-01_ses-01_dwi.bval
                            ├── sub-01_ses-01_dwi.bvec
                            └── sub-01_ses-01_dwi.nii.gz

    You can use AFQ.definitions.image.ImageFile to provide these custom scalars to the AFQ.api objects:

        ICVF_scalar = ImageFile(
          suffix="ICVF",
          filters={"scope": "noddi"})

        ODI_scalar = ImageFile(
          suffix="ODI",
          filters={"scope": "noddi"})

        api.GroupAFQ(
          "my_bids_path",
          scalars=["dti_fa", "dti_md", ICVF_scalar, ODI_scalar])

### 4. Outputs can be used as inputs to AFQ-Insight or AFQ-Browser
1. [AFQ-Insight](https://github.com/richford/AFQ-Insight) is an open source, python based statistical learning library for tractometry
        
2. [AFQ-Browser](https://yeatmanlab.github.io/AFQ-Browser/) is an open source, python based visualization library for tractometry
    
3. Outputs of Matlab based AFQ will also work for both of these

### 5. Using the pyAFQ configuration file
pyAFQ provides a [command-line interface (CLI)](https://yeatmanlab.github.io/pyAFQ/usage/index.html). After installing the software, and organizing the data, run:

    pyAFQ /path/to/config.toml

At minimum, the file should contain the BIDS path:

    [BIDS_PARAMS]
    bids_path = "path/to/study

But additional configuration options can be provided. See an example configuration file below:

    [BIDS_PARAMS]
    bids_path = "path/to/study
    preproc_pipeline = "qsiprep"
    bids_filters = "{'desc': 'preproc'}"

    [MAPPING]
    mapping_definition = "ItkMap(warp_suffix='xfm', warp_filters={'from': 'MNI152NLin2009cAsym', 'to': 'T1w', 'scope': 'qsiprep'})"

    [DATA]
    brain_mask_definition = "ImageFile(suffix='mask', filters={'desc': 'brain', 'space': 'T1w', 'scope': 'qsiprep'})"
    bundle_info = "['ATR_L', 'ATR_R', 'CGC_L', 'CGC_R', 'CST_L', 'CST_R', 'IFO_L', 'IFO_R', 'ILF_L', 'ILF_R', 'SLF_L', 'SLF_R', 'ARC_L', 'ARC_R', 'UNC_L', 'UNC_R', 'AntFrontal', 'Motor', 'Occipital', 'Orbital', 'PostParietal', 'SupFrontal', 'SupParietal', 'Temporal']"

    [SEGMENTATION]
    scalars = "['dki_fa', 'dki_md', 'dki_awf', 'dki_mk']"

    [TRACTOGRAPHY_PARAMS]
    seed_mask = "ScalarImage('dki_fa')"
    stop_mask = "ScalarImage('dki_fa')"
    odf_model = "CSD"
    directions = "prob"

    [SEGMENTATION_PARAMS]
    parallel_segmentation = "{'n_jobs': 1, 'engine': 'joblib', 'backend': 'loky'}"

## Section 4: Summary
    1. pyAFQ is free and open source

    2. pyAFQ is easy to install and run on the cloud

    3. pyAFQ uses BIDS datasets

    4. pyAFQ is highly customizable and integrates with other pipelines
  
Documentation: https://yeatmanlab.github.io/pyAFQ/ <br />
Code: https://github.com/yeatmanlab/pyAFQ <br />

# Thank you for listening!